In [1]:
from Core import Nanoparticle as NP 
import numpy as np

p = NP.Nanoparticle()
p.truncated_octahedron(6,1,{'Pt':1})

Resulting stoichiometry: {'Pt': 140}


In [48]:
from Core.GlobalFeatureClassifier import SimpleFeatureClassifier
from Core.GlobalFeatureClassifier import TopologicalFeatureClassifier

class LayererTopologicalDescriptors(SimpleFeatureClassifier):

    def __init__(self, symbols, particle):
        SimpleFeatureClassifier.__init__(self, symbols)
        self.feature_key = 'LTOP'       
        self.top =  TopologicalFeatureClassifier(symbols)
        self.layers = self.get_layer_indices(particle)
        self.n1, self.n2 = np.unique(particle.atoms.atoms.numbers)
    
    def compute_layers(self, particle):
        positions = particle.atoms.atoms.get_positions()
        x_layer, y_layer, z_layer = [np.unique(p.atoms.atoms.positions[:,coord]) for coord in range(3)]
        return x_layer, y_layer, z_layer

    def get_layer_indices(self, particle):
        positions = particle.atoms.atoms.get_positions()
        x_layer, y_layer, z_layer = self.compute_layers(particle)

        x_layer_indices = {x : np.where(positions[:,0] == x)[0] for x in x_layer}
        y_layer_indices = {y : np.where(positions[:,1] == y)[0] for y in y_layer}
        z_layer_indices = {z : np.where(positions[:,2] == z)[0] for z in z_layer}

        return x_layer_indices, y_layer_indices, z_layer_indices

    def get_layers_occupacy(self, particle):
        x_layer_indices, y_layer_indices, z_layer_indices = self.layers

        layers_occupacy_x = 0
        layers_occupacy_y = 0
        layers_occupacy_z = 0

        for layer in x_layer_indices.values():
            n_a = np.where(p.atoms.atoms.numbers[layer]==self.n1)[0]
            n_b = np.where(p.atoms.atoms.numbers[layer]==self.n2)[0]
            layers_occupacy_x += abs(len(n_a) - len(n_b))

        for layer in y_layer_indices.values():
            n_a = np.where(p.atoms.atoms.numbers[layer]==self.n1)[0]
            n_b = np.where(p.atoms.atoms.numbers[layer]==self.n2)[0]
            layers_occupacy_y += abs(len(n_a) - len(n_b))

        for layer in z_layer_indices.values():
            n_a = np.where(p.atoms.atoms.numbers[layer]==self.n1)[0]
            n_b = np.where(p.atoms.atoms.numbers[layer]==self.n2)[0]
            layers_occupacy_z += abs(len(n_a) - len(n_b))

        return np.array([layers_occupacy_x, layers_occupacy_y, layers_occupacy_z])

    def compute_feature_vector(self, particle):
        self.top.compute_feature_vector(particle)
        layericity = np.array(self.get_layers_occupacy(particle))
        return np.append(particle.get_feature_vector(self.top.feature_key), layericity)
            

In [2]:
from Core.GlobalFeatureClassifier import LayererTopologicalDescriptors
from Core import Nanoparticle as NP 

p = NP.Nanoparticle()
p.read('/home/riccardo/Scientific_Projects/ETOP_paper_figures/xyz/Pd72Zn68_ETOP.xyz')
#p.random_ordering({'Pt':69,'Au':71})
ltop = LayererTopologicalDescriptors(p.get_all_symbols(), p)
ltop.compute_feature_vector(p)
p.get_feature_vector('LTOP')

array([108., 104., 424.,   0.,   0.,   0.,   0.,   0.,   0.,  16.,  16.,
         0.,  16.,   0.,   0.,  24.,   0.,   0.,   0.,   0.,   0.,   0.,
         8.,   8.,   0.,  32.,   0.,   0.,  20., 140.,   4.,   4.])

In [61]:
feature_vector = np.empty(32)
feature_vector[:29] 
feature_vector[29:] 

array([6.92770194e-310, 6.92770194e-310, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 5.02034658e+175, 3.92841766e-061, 1.91620945e-076,
       1.58821150e-047, 6.23174874e+174, 3.40473612e-057, 2.16385130e+190,
       1.94918964e-153, 5.30581644e+180, 4.32453723e-096, 2.14027814e+161,
       8.67509195e+199, 4.31536478e-096, 4.82337723e+228, 4.79408759e+174,
       1.10684325e-047, 9.95692492e-043, 3.50295081e-033, 4.26032547e-096,
       1.16097020e-028, 8.26219736e-072, 2.54867591e-028, 5.93051569e-038,
       1.04917183e-153, 9.08366791e+223, 6.14415221e-144, 6.17582057e-322])

In [43]:
a, b, = np.unique(p.atoms.atoms.numbers)

In [44]:
a

30

In [30]:
p.atoms.atoms.positions

array([[-7.8 , -1.95,  0.  ],
       [-7.8 , -0.  , -1.95],
       [-7.8 , -0.  ,  1.95],
       [-7.8 ,  1.95,  0.  ],
       [-5.85, -3.9 ,  0.  ],
       [-3.9 , -5.85,  0.  ],
       [-3.9 , -3.9 , -1.95],
       [-3.9 , -3.9 ,  1.95],
       [-5.85, -0.  , -3.9 ],
       [-3.9 , -1.95, -3.9 ],
       [-3.9 , -0.  , -5.85],
       [-5.85, -1.95, -1.95],
       [-5.85, -0.  ,  0.  ],
       [-3.9 , -1.95,  0.  ],
       [-3.9 , -0.  , -1.95],
       [-5.85, -1.95,  1.95],
       [-5.85, -0.  ,  3.9 ],
       [-3.9 , -1.95,  3.9 ],
       [-3.9 , -0.  ,  1.95],
       [-3.9 , -0.  ,  5.85],
       [-3.9 ,  1.95, -3.9 ],
       [-5.85,  1.95, -1.95],
       [-5.85,  3.9 ,  0.  ],
       [-3.9 ,  1.95,  0.  ],
       [-3.9 ,  3.9 , -1.95],
       [-5.85,  1.95,  1.95],
       [-3.9 ,  1.95,  3.9 ],
       [-3.9 ,  3.9 ,  1.95],
       [-3.9 ,  5.85,  0.  ],
       [-1.95, -7.8 ,  0.  ],
       [ 0.  , -7.8 , -1.95],
       [ 0.  , -7.8 ,  1.95],
       [-1.95, -3.9 , -3.9 ],
       [ 0

In [126]:
a1 = np.array([10,20])
a2 = np.array([20])
np.append(a1,a2)

array([10, 20, 20])

In [76]:
ltop = LayererTopologicalDescriptors(['Pt'])
x_layer_indices, _, _ = ltop.get_layer_indices(p)

#layers_occupacy = ltop.get_layers_occupacy(p)
for x in x_layer_indices.values():
    print(x)


[0 1 2 3]
[ 4  8 11 12 15 16 21 22 25]
[ 5  6  7  9 10 13 14 17 18 19 20 23 24 26 27 28]
[29 32 35 36 39 40 44 46 47 50 51 54 55 58 59 63 64 67 68 71 72 75 79 80
 83]
[30 31 33 34 37 38 41 42 43 45 48 49 52 53 56 57 60 61 62 65 66 69 70 73
 74 76 77 78 81 82 84 85]
[ 86  87  88  89  92  93  95  96  97 100 101 104 105 108 109 111 112 114
 115 118 119 122 123 124 126]
[ 90  91  94  98  99 102 103 106 107 110 113 116 117 120 121 125]
[127 128 129 130 133 134 136 137 139]
[131 132 135 138]


In [80]:
p.atoms.atoms.numbers[x]/18

array([4.33333333, 4.33333333, 4.33333333, 4.33333333])